In [9]:
import os
from openai import OpenAI
from evaluate import load
from transformers import WhisperProcessor
from evaluate import load
import json
import numpy as np

wer = load("wer")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")

In [103]:
client = OpenAI(
  organization='org-cQcini8CmgApuWQd7TW63AbZ',
  project='proj_l5hBsnWNcaXmXjNkaNabPmDw',
  api_key = 'sk-proj-Jzb7ixiKOmmUDVdYYenIT3BlbkFJyZA4dzrp7BFVO4fU1dvg'
)


audio_file= open("/Users/raghavgarg/Desktop/CS224S/Data/Audio Recordings/CAR0001.mp3", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file,
  response_format="verbose_json",
  timestamp_granularities=["word"]
)
print(transcription)

Transcription(text="what brought you in today? Sure, I'm just having a lot of chest pain and so I thought I should get it checked out. Okay, and before we start could you remind me of your gender and age? Sure, I'm 39. I'm a male. Okay, and so when did this chest pain start? It started last night but it's becoming sharper. Okay, and where is this pain located? It's located on the left side of my chest. Okay, and so how long has it been going on for than if it started last night? So I guess it would be a couple hours now, maybe like eight. Okay, has it been constant throughout that time or changing? I would say it's been pretty constant, yeah. Okay, and how would you describe the pain? People will use words sometimes like sharp, burning, achy. I'd say it's pretty sharp, yeah. Sharp, okay. Anything that you have done, tried since last night that's made the pain better? Not laying down helps. Okay, so do you find laying down makes the pain worse? Yes, definitely. Okay, do you find that th

In [104]:
system_prompt = '''You are a helpful assistant for this speaker diarisation task. Your task is to indicate when the Doctor is speaking with D and the patient speaking with P. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided. For example: D: How may I help you?

P: Hi, yes it's nice to meet you. I've been having this cough that's been going on for the last few days and I have had difficulty breathing.

D: Oh, I'm so sorry to hear that. When did this start?

P: Oh, so the cough started about'''

def generate_corrected_transcript(system_prompt, transcription_obj):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": transcription_obj.text
            }
        ]
    )
    return response

response = generate_corrected_transcript(0, system_prompt, transcription)

In [113]:
def compute_wer(batch):
    def parse_transcript(transcript):
        speaker_segments, d_speaker_segments, p_speaker_segments = [], [], []
        lines = transcript.split('\n')

        for line in lines:
            if ':' in line:
                speaker, text = line.split(':', 1)
                text = processor.tokenizer.normalize(text)
                words = text.strip().split()
                if speaker == 'D':
                    d_speaker_segments += [word for word in words]
                if speaker == 'P':
                    p_speaker_segments += [word for word in words]
                speaker_segments += [word for word in words]
        return ' '.join(speaker_segments), ' '.join(d_speaker_segments), ' '.join(p_speaker_segments)

    batch["pred_norm"], batch["pred_norm_d"], batch["pred_norm_p"] = parse_transcript(batch["pred"])
    batch["ref_norm"], batch["ref_norm_d"], batch["ref_norm_p"] = parse_transcript(batch["ref"])
    batch["wer"] = wer.compute(predictions=[batch["pred_norm"]], references=[batch["ref_norm"]])
    wer_dd = wer.compute(predictions=[batch["pred_norm_d"]], references=[batch["ref_norm_d"]])
    wer_pp = wer.compute(predictions=[batch["pred_norm_p"]], references=[batch["ref_norm_p"]])
    wer_dp = wer.compute(predictions=[batch["pred_norm_d"]], references=[batch["ref_norm_p"]])
    wer_pd = wer.compute(predictions=[batch["pred_norm_p"]], references=[batch["ref_norm_d"]])
    batch["cpWER"] = min(wer_dd, wer_pp, wer_dp, wer_pd)
    return batch

candidate_transcript = response.choices[0].message.content
with open("/Users/raghavgarg/Desktop/CS224S/Data/Clean Transcripts/CAR0001.txt", 'r') as f:
    reference_transcript = f.read()

example = [{"ref": reference_transcript, "pred": candidate_transcript}]
result = list(map(compute_wer, example))

In [115]:
result

[{'ref': "D: What brought you in today?\n\nP: Sure, I'm I'm just having a lot of chest pain and and so I thought I should get it checked out.\n\nD: OK, before we start, could you remind me of your gender and age? \n\nP: Sure 39, I'm a male.\n\nD: OK, and so when did this chest pain start?\n\nP: It started last night, but it's becoming sharper.\n\nD: OK, and where is this pain located? \n\nP: It's located on the left side of my chest.\n\nD: OK, and, so how long has it been going on for then if it started last night?\n\nP: So I guess it would be a couple of hours now, maybe like 8.\n\nD: OK. Has it been constant throughout that time, or uh, or changing? \n\nP: I would say it's been pretty constant, yeah.\n\nD: OK, and how would you describe the pain? People will use words sometimes like sharp, burning, achy. \n\nP: I'd say it's pretty sharp, yeah.\n\nD: Sharp OK. Uh, anything that you have done tried since last night that's made the pain better?\n\nP: Um not laying down helps.\n\nD: OK, 

In [7]:
# Paths
audio_folder = '/Users/raghavgarg/Desktop/CS224S/Data/Audio Recordings'
clean_folder = '/Users/raghavgarg/Desktop/CS224S/Data/Clean Transcripts'
whisper_transcripts_folder = '/Users/raghavgarg/Desktop/CS224S/Data/whisper_transcripts'
wer_transcripts_folder = '/Users/raghavgarg/Desktop/CS224S/Data/wer_transcripts'

if not os.path.exists(whisper_transcripts_folder):
    os.makedirs(whisper_transcripts_folder)

if not os.path.exists(wer_transcripts_folder):
    os.makedirs(wer_transcripts_folder)

# System prompt for speaker diarization correction
system_prompt = '''You are a helpful assistant for this speaker diarisation task. Your task is to indicate when the Doctor is speaking with D and the patient speaking with P. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided. For example:
D: How may I help you?
P: Hi, yes it's nice to meet you. I've been having this cough that's been going on for the last few days and I have had difficulty breathing.
D: Oh, I'm so sorry to hear that. When did this start?
P: Oh, so the cough started about'''

client = OpenAI(
  organization='org-cQcini8CmgApuWQd7TW63AbZ',
  project='proj_l5hBsnWNcaXmXjNkaNabPmDw',
  api_key = 'sk-proj-Jzb7ixiKOmmUDVdYYenIT3BlbkFJyZA4dzrp7BFVO4fU1dvg'
)

def transcribe_audio(file_path):
    with open(file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        response_format="verbose_json",
        timestamp_granularities=["word"]
        )
    return transcription

def generate_corrected_transcript(system_prompt, transcription_obj):

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": transcription_obj.text}
        ]
    )
    return response

def parse_transcript(transcript):
    speaker_segments, d_speaker_segments, p_speaker_segments = [], [], []
    lines = transcript.split('\n')

    for line in lines:
        if ':' in line:
            speaker, text = line.split(':', 1)
            text = processor.tokenizer.normalize(text)
            words = text.strip().split()
            if speaker == 'D':
                d_speaker_segments += [word for word in words]
            if speaker == 'P':
                p_speaker_segments += [word for word in words]
            speaker_segments += [word for word in words]
    return ' '.join(speaker_segments), ' '.join(d_speaker_segments), ' '.join(p_speaker_segments)

def compute_wer(batch):
    batch["pred_norm"], batch["pred_norm_d"], batch["pred_norm_p"] = parse_transcript(batch["pred"])
    batch["ref_norm"], batch["ref_norm_d"], batch["ref_norm_p"] = parse_transcript(batch["ref"])
    batch["wer"] = wer.compute(predictions=[batch["pred_norm"]], references=[batch["ref_norm"]])
    batch["wer_dd"] = wer.compute(predictions=[batch["pred_norm_d"]], references=[batch["ref_norm_d"]])
    batch["wer_pp"] = wer.compute(predictions=[batch["pred_norm_p"]], references=[batch["ref_norm_p"]])
    batch["wer_dp"] = wer.compute(predictions=[batch["pred_norm_d"]], references=[batch["ref_norm_p"]])
    batch["wer_pd"] = wer.compute(predictions=[batch["pred_norm_p"]], references=[batch["ref_norm_d"]])
    batch["cpWER"] = min(batch["wer_dd"], batch["wer_pp"], batch["wer_dp"], batch["wer_pd"])
    return batch

results = []
for filename in os.listdir(audio_folder):
    finished_files = os.listdir(wer_transcripts_folder)
    if filename.replace('.mp3', '.json') in finished_files:
        print(f"Skipping {filename} as it has already been processed.")
        continue
    print(f"Processing {filename}...")
    if filename.endswith('.mp3'):
        audio_path = os.path.join(audio_folder, filename)
        reference_path = os.path.join(clean_folder, filename.replace('.mp3', '.txt'))

        with open(reference_path, 'r') as f:
            reference_transcript = f.read()

        transcription = transcribe_audio(audio_path)
        whisper_transcription_path = os.path.join(
            whisper_transcripts_folder, filename.replace('.mp3', '.json'))
        with open(whisper_transcription_path, 'w') as f:
            json.dump({
                "text": transcription.text,
                "duration": transcription.duration,
                "words": transcription.words
            }, f, indent=4)

        response = generate_corrected_transcript(system_prompt, transcription)
        candidate_transcript = response.choices[0].message.content

        example = {"ref": reference_transcript, "pred": candidate_transcript}
        result = compute_wer(example)
        result['filename'] = filename.replace('.mp3', '')
        results.append(result)

        wer_transcription_path = os.path.join(
            wer_transcripts_folder, filename.replace('.mp3', '.json'))
        with open(wer_transcription_path, 'w') as f:
            json.dump(result, f, indent=4)

print("Processing complete.")


Skipping RES0116.mp3 as it has already been processed.
Skipping RES0102.mp3 as it has already been processed.
Skipping RES0062.mp3 as it has already been processed.
Skipping RES0076.mp3 as it has already been processed.
Skipping RES0089.mp3 as it has already been processed.
Skipping RES0088.mp3 as it has already been processed.
Skipping RES0077.mp3 as it has already been processed.
Skipping RES0063.mp3 as it has already been processed.
Skipping CAR0001.mp3 as it has already been processed.
Skipping RES0103.mp3 as it has already been processed.
Skipping RES0117.mp3 as it has already been processed.
Skipping RES0101.mp3 as it has already been processed.
Skipping RES0129.mp3 as it has already been processed.
Skipping CAR0003.mp3 as it has already been processed.
Skipping RES0075.mp3 as it has already been processed.
Skipping RES0061.mp3 as it has already been processed.
Skipping RES0049.mp3 as it has already been processed.
Skipping RES0048.mp3 as it has already been processed.
Skipping R

In [10]:
wer_values = []
wer_dd_values = []
wer_pp_values = []
wer_dp_values = []
wer_pd_values = []
cpWER_values = []

# Process each JSON file in the WER transcripts folder
for filename in os.listdir(wer_transcripts_folder):
    if filename.endswith('.json'):
        file_path = os.path.join(wer_transcripts_folder, filename)

        # Load the JSON file
        with open(file_path, 'r') as f:
            data = json.load(f)

        # Extract WER values
        wer_values.append(data['wer'])
        wer_dd_values.append(data['wer_dd'])
        wer_pp_values.append(data['wer_pp'])
        wer_dp_values.append(data['wer_dp'])
        wer_pd_values.append(data['wer_pd'])
        cpWER_values.append(data['cpWER'])

# Compute mean values for each WER type
mean_wer = np.mean(wer_values)
mean_wer_dd = np.mean(wer_dd_values)
mean_wer_pp = np.mean(wer_pp_values)
mean_wer_dp = np.mean(wer_dp_values)
mean_wer_pd = np.mean(wer_pd_values)
mean_cpWER = np.mean(cpWER_values)

# Print the mean values
print(f"Mean WER: {mean_wer}")
print(f"Mean WER (DD): {mean_wer_dd}")
print(f"Mean WER (PP): {mean_wer_pp}")
print(f"Mean WER (DP): {mean_wer_dp}")
print(f"Mean WER (PD): {mean_wer_pd}")
print(f"Mean cpWER: {mean_cpWER}")

Mean WER: 0.12476473620318158
Mean WER (DD): 0.171657548026743
Mean WER (PP): 0.15815551685786855
Mean WER (DP): 1.2518275046564282
Mean WER (PD): 0.9434873523777008
Mean cpWER: 0.12034451244583716
